In [0]:
layer1_base_path = "abfss://layer-1@projectonsaleslt.dfs.core.windows.net"
layer2_base_path = "abfss://layer-2@projectonsaleslt.dfs.core.windows.net"


In [0]:
table_name = []

for item in dbutils.fs.ls(f"{layer1_base_path}/SalesLT/"):
    table_name.append(item.name.replace("/", ""))

print(table_name)


['Address', 'Customer', 'CustomerAddress', 'Product', 'ProductCategory', 'ProductDescription', 'ProductModel', 'ProductModelProductDescription', 'SalesOrderDetail', 'SalesOrderHeader']


In [0]:
from pyspark.sql.functions import col, date_format, to_timestamp
from pyspark.sql.types import TimestampType

for i in table_name:
    input_path = f"{layer1_base_path}/SalesLT/{i}/{i}.parquet"
    
    df = spark.read.format("parquet").load(input_path)
    
    for col_name in df.columns:
        if "date" in col_name.lower():
            df = df.withColumn(
                col_name,
                date_format(
                    to_timestamp(col(col_name).cast(TimestampType())),
                    "dd-MMM-yyyy"
                )
            )

    output_path = f"{layer2_base_path}/SalesLT/{i}/"
    
    df.write.format("delta") \
      .mode("overwrite") \
      .save(output_path)
